In [ ]:
# Setup: imports and display options
import pandas as pd
import numpy as np

pd.options.display.max_columns = 50
pd.options.display.width = 120
pd.options.display.max_rows = 20

print("Setup complete!")
print(f"pandas version: {pd.__version__}")

## 1. Load Data

We'll use the media contacts data which has many columns for different platforms:

In [ ]:
# Load and prepare data
media_df = pd.read_csv('../data/media_contacts.csv')
demo_df = pd.read_csv('../data/socio_demos.csv')

# Standardize column names
media_df.columns = media_df.columns.str.strip().str.lower().str.replace(' ', '_')
demo_df.columns = demo_df.columns.str.strip().str.lower().str.replace(' ', '_')

print(f"Media shape: {media_df.shape}")
print(f"\nMedia columns:")
print(media_df.columns.tolist())

In [ ]:
# Look at the structure
print("Sample data (first 3 rows, key columns):")
sample_cols = ['person_id', 'tv_total', 'online_total', 'print_total', 
               'youtube_mobile', 'youtube_desktop', 'youtube_tablet', 'youtube_ctv']
print(media_df[sample_cols].head(3))

## 2. Understanding Wide vs Long Format

### Wide Format (Current)

Each variable has its own column. Good for human reading:

```
person_id | tv_total | online_total | print_total
----------|----------|--------------|------------
1         | 45       | 23           | 12
2         | 67       | 34           | 8
```

### Long Format (After Melt)

Variables are stacked. Good for analysis and plotting:

```
person_id | channel  | exposure
----------|----------|----------
1         | tv       | 45
1         | online   | 23
1         | print    | 12
2         | tv       | 67
2         | online   | 34
2         | print    | 8
```

## 3. Melt: Wide to Long

`pd.melt()` converts wide format to long format.

### SPSS/R Analogies:
- **SPSS**: `VARSTOCASES` command
- **R tidyr**: `pivot_longer()` or older `gather()`
- **R reshape2**: `melt()`

In [ ]:
# Simple melt example with total exposure columns
totals_long = pd.melt(
    media_df,
    id_vars=['person_id'],  # Keep these columns
    value_vars=['tv_total', 'online_total', 'print_total'],  # Melt these
    var_name='channel',  # Name for the variable column
    value_name='exposure'  # Name for the value column
)

print("After melting total exposure columns:")
print(totals_long.head(10))
print(f"\nShape: {totals_long.shape}")

In [ ]:
# Check: each person should have 3 rows now
print(f"\nOriginal rows: {len(media_df)}")
print(f"After melt: {len(totals_long)}")
print(f"Expected: {len(media_df) * 3}")

# Verify
person_check = totals_long[totals_long['person_id'] == 1]
print(f"\nPerson ID 1 has {len(person_check)} rows:")
print(person_check)

### Cleaning Up Channel Names

In [ ]:
# Remove "_total" suffix for cleaner names
totals_long['channel'] = totals_long['channel'].str.replace('_total', '')

print("Cleaned channel names:")
print(totals_long.head(10))

## 4. YouTube Metrics: Real-World Scenario

Let's analyze YouTube viewing across devices (Mobile, Desktop, Tablet, CTV):

In [ ]:
# Identify YouTube columns
youtube_cols = [col for col in media_df.columns if col.startswith('youtube_')]
print(f"YouTube columns ({len(youtube_cols)}):")
print(youtube_cols)

In [ ]:
# Melt YouTube metrics to long format
youtube_long = pd.melt(
    media_df,
    id_vars=['person_id', 'purchase'],  # Keep ID and purchase
    value_vars=youtube_cols,
    var_name='device',
    value_name='minutes_viewed'
)

print("YouTube data in long format:")
print(youtube_long.head(12))
print(f"\nShape: {youtube_long.shape}")

In [ ]:
# Clean device names
youtube_long['device'] = youtube_long['device'].str.replace('youtube_', '').str.capitalize()

print("\nCleaned device names:")
print(youtube_long['device'].value_counts())

In [ ]:
# Now we can easily analyze by device
device_summary = youtube_long.groupby('device').agg(
    total_minutes=('minutes_viewed', 'sum'),
    avg_minutes=('minutes_viewed', 'mean'),
    median_minutes=('minutes_viewed', 'median'),
    users_with_viewing=('minutes_viewed', lambda x: (x > 0).sum())
).round(2)

print("\nYouTube viewing by device:")
print(device_summary.sort_values('avg_minutes', ascending=False))

## 5. Melt with Multiple ID Variables

Keep multiple identifier columns:

In [ ]:
# Merge with demo first
merged_df = pd.merge(media_df, demo_df, on='person_id', how='inner')

# Melt keeping demographic info
youtube_demo = pd.melt(
    merged_df,
    id_vars=['person_id', 'gender', 'purchase'],
    value_vars=youtube_cols,
    var_name='device',
    value_name='minutes_viewed'
)

youtube_demo['device'] = youtube_demo['device'].str.replace('youtube_', '').str.capitalize()

print("YouTube with demographics:")
print(youtube_demo.head(10))

In [ ]:
# Analyze by gender and device
gender_device = youtube_demo.groupby(['gender', 'device']).agg(
    avg_minutes=('minutes_viewed', 'mean'),
    users=('person_id', 'nunique')
).round(2)

print("\nYouTube viewing by gender and device:")
print(gender_device)

## 6. Pivot: Long to Wide

`pivot()` converts long format back to wide format.

### SPSS/R Analogies:
- **SPSS**: `CASESTOVARS` command
- **R tidyr**: `pivot_wider()` or older `spread()`
- **R reshape2**: `dcast()`

In [ ]:
# Start with long format (youtube_long)
print("Long format (first 12 rows):")
print(youtube_long.head(12))

In [ ]:
# Pivot back to wide format
youtube_wide = youtube_long.pivot(
    index='person_id',  # Rows
    columns='device',    # New columns
    values='minutes_viewed'  # Values to fill
)

print("\nPivoted back to wide format:")
print(youtube_wide.head())
print(f"\nShape: {youtube_wide.shape}")

In [ ]:
# Reset index to make person_id a regular column
youtube_wide_reset = youtube_wide.reset_index()

print("\nWith person_id as a column:")
print(youtube_wide_reset.head())

## 7. Pivot Table: Aggregating Values

`pivot_table()` aggregates when you have duplicate combinations:

In [ ]:
# Create scenario with duplicates
# Add age band to youtube_demo
merged_df['age'] = 2025 - pd.to_datetime(
    merged_df['birthday'].astype(int).astype(str), 
    format='%Y%m%d'
).dt.year

merged_df['age_band'] = pd.cut(
    merged_df['age'],
    bins=[0, 25, 35, 45, 55, 100],
    labels=['<25', '25-34', '35-44', '45-54', '55+']
)

youtube_demo2 = pd.melt(
    merged_df,
    id_vars=['person_id', 'gender', 'age_band', 'purchase'],
    value_vars=youtube_cols,
    var_name='device',
    value_name='minutes_viewed'
)
youtube_demo2['device'] = youtube_demo2['device'].str.replace('youtube_', '').str.capitalize()

print("Sample with age_band:")
print(youtube_demo2.head(10))

In [ ]:
# Pivot table: average viewing by age_band x device
viewing_pivot = youtube_demo2.pivot_table(
    index='age_band',
    columns='device',
    values='minutes_viewed',
    aggfunc='mean'  # Aggregate function
)

print("\nAverage YouTube viewing by age band and device:")
print(viewing_pivot.round(2))

In [ ]:
# Multiple aggregations
viewing_pivot_multi = youtube_demo2.pivot_table(
    index='age_band',
    columns='device',
    values='minutes_viewed',
    aggfunc=['mean', 'median', 'count']
)

print("\nMultiple aggregations:")
print(viewing_pivot_multi.round(2))

## 8. Difference Between pivot() and pivot_table()

| Feature | `pivot()` | `pivot_table()` |
|---------|-----------|-----------------|
| Duplicates | ❌ Raises error | ✅ Aggregates |
| Aggregation | ❌ No | ✅ Yes (mean, sum, etc.) |
| Multiple values | ❌ No | ✅ Yes |
| Speed | ⚡ Faster | 🐢 Slower |
| Use when | Unique combinations | Duplicates expected |

In [ ]:
# Demonstrate pivot() error with duplicates
# (This would error - commented out)
# youtube_demo2.pivot(index='age_band', columns='device', values='minutes_viewed')
# ValueError: Index contains duplicate entries, cannot reshape

# But pivot_table() handles it:
result = youtube_demo2.pivot_table(
    index='age_band',
    columns='device',
    values='minutes_viewed',
    aggfunc='mean'
)
print("pivot_table() handles duplicates by aggregating:")
print(result.round(2))

## 9. Multi-Index After Pivot

Pivoting can create multi-index DataFrames:

In [ ]:
# Create multi-index pivot
multi_pivot = youtube_demo2.pivot_table(
    index=['age_band', 'gender'],
    columns='device',
    values='minutes_viewed',
    aggfunc='mean'
)

print("Multi-index pivot:")
print(multi_pivot.round(2))
print(f"\nIndex levels: {multi_pivot.index.nlevels}")

In [ ]:
# Reset index to flatten
multi_flat = multi_pivot.reset_index()

print("\nFlattened multi-index:")
print(multi_flat.head(10))

In [ ]:
# Access multi-index data
print("\nAccess specific rows:")
print(multi_pivot.loc[('25-34', 'Male'), :])

print("\nAccess all ages for Female:")
print(multi_pivot.loc[(slice(None), 'Female'), :])

## 10. Stack and Unstack

Alternative reshaping methods for multi-index:

In [ ]:
# Start with wide format
sample_wide = media_df[['person_id', 'tv_total', 'online_total', 'print_total']].head(5)
sample_wide = sample_wide.set_index('person_id')

print("Wide format:")
print(sample_wide)

In [ ]:
# Stack: columns to rows
stacked = sample_wide.stack()

print("\nStacked (Series):")
print(stacked)
print(f"\nType: {type(stacked)}")

In [ ]:
# Unstack: rows back to columns
unstacked = stacked.unstack()

print("\nUnstacked (back to wide):")
print(unstacked)

In [ ]:
# Stack/unstack with multi-index
multi_example = youtube_demo2.groupby(['age_band', 'gender', 'device'])['minutes_viewed'].mean()

print("Multi-index Series:")
print(multi_example.head(20))

In [ ]:
# Unstack last level (device)
unstacked_device = multi_example.unstack(level='device')

print("\nUnstacked device level:")
print(unstacked_device.head(10))

## 11. Real-World Example: Preparing for Visualization

Often need long format for plotting libraries (seaborn, plotly):

In [ ]:
# Prepare data for visualization
# Goal: Compare TV, Online, Print exposure by age band

# Melt total columns
viz_data = pd.melt(
    merged_df,
    id_vars=['person_id', 'age_band', 'gender', 'purchase'],
    value_vars=['tv_total', 'online_total', 'print_total'],
    var_name='channel',
    value_name='exposure'
)

viz_data['channel'] = viz_data['channel'].str.replace('_total', '').str.capitalize()

print("Data ready for plotting:")
print(viz_data.head(15))

In [ ]:
# Aggregate for cleaner visualization
viz_summary = viz_data.groupby(['age_band', 'channel'], observed=True).agg(
    avg_exposure=('exposure', 'mean'),
    median_exposure=('exposure', 'median'),
    sample_size=('person_id', 'count')
).reset_index()

print("\nSummary ready for plotting:")
print(viz_summary)

In [ ]:
# This format works great with seaborn or plotly:
# import seaborn as sns
# sns.barplot(data=viz_summary, x='age_band', y='avg_exposure', hue='channel')

print("\n✅ This long format is perfect for:")
print("  - seaborn: sns.barplot(), sns.lineplot(), sns.boxplot()")
print("  - plotly: px.bar(), px.line(), px.scatter()")
print("  - matplotlib: grouped plotting")

## 12. Complex Reshaping: Multiple Value Columns

Melt multiple value columns simultaneously:

In [ ]:
# Create subset with purchase info
sample_reshape = merged_df[['person_id', 'gender', 
                             'youtube_mobile', 'youtube_desktop', 
                             'youtube_tablet', 'youtube_ctv', 'purchase']].head(10)

print("Original data:")
print(sample_reshape)

In [ ]:
# Melt YouTube columns
reshaped = pd.melt(
    sample_reshape,
    id_vars=['person_id', 'gender', 'purchase'],
    value_vars=['youtube_mobile', 'youtube_desktop', 'youtube_tablet', 'youtube_ctv'],
    var_name='device',
    value_name='minutes'
)

reshaped['device'] = reshaped['device'].str.replace('youtube_', '').str.capitalize()

print("\nReshaped:")
print(reshaped.head(20))

## 13. When to Use Each Method

### Use `melt()` when:
- Converting wide to long format
- Preparing for visualization
- Need tidy data structure
- Multiple metrics per observation

### Use `pivot()` when:
- Converting long to wide
- No duplicate combinations
- Need simple reshape
- Performance is critical

### Use `pivot_table()` when:
- Converting long to wide
- Have duplicate combinations
- Need aggregation
- Creating summary tables

### Use `stack()`/`unstack()` when:
- Working with multi-index
- Need fine control over levels
- Reshaping hierarchical data

## Summary

In this notebook, you learned:

✅ Reshape wide to long with `melt()`  
✅ Reshape long to wide with `pivot()` and `pivot_table()`  
✅ Handle multi-index DataFrames  
✅ Use `stack()` and `unstack()` for multi-level reshaping  
✅ SPSS/R reshaping analogies  
✅ Prepare data for visualization  
✅ Analyze YouTube metrics by device  
✅ Choose the right reshaping tool  

### Key Takeaways

1. **Long format**: Better for analysis and plotting
2. **Wide format**: Better for human reading
3. **melt() is your friend**: Master it for data prep
4. **pivot() vs pivot_table()**: Use pivot() when no duplicates
5. **Multi-index**: Learn to flatten with reset_index()
6. **Visualization libraries prefer long format**: seaborn, plotly, etc.

### Next Steps

In the next notebook (**07_exporting_and_saving.ipynb**), we'll:
- Export to CSV, Parquet, Excel
- Choose the right format
- Use compression effectively
- Save analysis results
- Create reproducible workflows

## 🎯 Practice Exercises

Try these on your own:

1. Melt all social media columns (Facebook, Instagram, Twitter, etc.) to long format
2. Create a pivot table showing average TV exposure by gender x age_band
3. Reshape youtube_long back to wide format using pivot()
4. Use stack() on a DataFrame with 3 numeric columns
5. Create a visualization-ready dataset with all channel totals in long format
6. Calculate the percent of total viewing for each YouTube device
7. Create a multi-index pivot with age_band, gender as rows and device as columns
8. Use unstack() to convert a grouped Series to a DataFrame

### Bonus Challenges

9. Melt both YouTube columns AND Facebook columns separately, then combine them
10. Create a pivot table with multiple aggregations (mean, median, count) and flatten the column names
11. Build a function that takes a DataFrame and list of columns, returns melted long format
12. Create a "heatmap-ready" pivot table of purchase rate by age_band x gender, then visualize it (if you have matplotlib/seaborn)

## Loading/Saving Data Between Notebooks

### Load Merged Data

If you saved merged data in previous notebooks:

```python
# Uncomment to load previously processed data
# merged_df = pd.read_csv('../outputs/merged_data.csv')
# 
# # Or load from Parquet (faster)
# merged_df = pd.read_parquet('../outputs/merged_data.parquet')
#
# print(f"Loaded merged data: {merged_df.shape}")
```

### Save Reshaped Data

Save long-format data for visualization:

```python
# Uncomment to save reshaped data
# import os
# os.makedirs('../outputs', exist_ok=True)
#
# # Save YouTube long format
# youtube_long.to_csv('../outputs/youtube_long.csv', index=False)
#
# # Save visualization-ready data
# viz_summary.to_csv('../outputs/viz_ready.csv', index=False)
#
# # Save pivot tables
# viewing_pivot.to_csv('../outputs/viewing_by_age_device.csv')
#
# print("Reshaped data saved!")
```